In [1]:
!pip install torch
import torch

In [2]:
!pip install torchtext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.4 MB/s eta 0:00:00


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [25]:
def load_dataset(filepath):
    texts = []
    labels = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            split_line = line.strip().split('\t')
            if len(split_line) == 2:
                texts.append(split_line[0])
                try:
                    labels.append(int(split_line[1]))
                except ValueError:
                    print(f"Invalid label in line: {line}")
    return texts, labels


In [26]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [27]:
# Correctly identify the text and label columns for train and test data
text_column_train = train_df.columns[2]
label_column_train = train_df.columns[0]
text_column_test = test_df.columns[1]
label_column_test = test_df.columns[0]

# Access the data using the column names
train_texts = train_df[text_column_train].tolist()
train_labels = train_df[label_column_train].tolist()
test_texts = test_df[text_column_test].tolist()
test_labels = test_df[label_column_test].tolist()

In [28]:
#Processing
# Tokenization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_VOCAB_SIZE = 20000
MAX_SEQUENCE_LENGTH = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

In [29]:
# Padding
train_padded = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [30]:
# Label encoding
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

In [31]:
# Convert labels to tensors
y_train = torch.tensor(train_labels_encoded, dtype=torch.long)
y_test = torch.tensor(test_labels_encoded, dtype=torch.long)

In [ ]:
# Load GloVe Embeddings
print("Loading GloVe embeddings...")
EMBEDDING_DIM = 100
glove_path = "/content/glove.6B.100d.txt"

embeddings_index = {}
with open(glove_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print("Preparing embedding matrix...")
embedding_matrix = np.zeros((MAX_VOCAB_SIZE, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    if i < MAX_VOCAB_SIZE:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

embedding_matrix = torch.tensor(embedding_matrix, dtype=torch.float)

In [33]:
# === Step 4: Build GRU Model ===
class GRUClassifier(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, output_dim, dropout):
        super(GRUClassifier, self).__init__()
        num_embeddings, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=True)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)
        _, hidden = self.gru(x)  # GRU only needs hidden state
        x = self.fc(self.dropout(hidden[-1]))
        return x

In [34]:
# Parameters
HIDDEN_DIM = 128
OUTPUT_DIM = len(set(train_labels))  # Number of classes in the dataset
DROPOUT = 0.5

model = GRUClassifier(embedding_matrix, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

In [35]:
# === Step 5: Train Model ===
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Create DataLoaders
train_sequences_tensor = torch.tensor(train_padded, dtype=torch.long)
test_sequences_tensor = torch.tensor(test_padded, dtype=torch.long)

train_dataset = TensorDataset(train_sequences_tensor, y_train)
test_dataset = TensorDataset(test_sequences_tensor, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def train(model, loader):
    model.train()
    total_loss = 0
    for sequences, labels in loader:
        optimizer.zero_grad()
        output = model(sequences)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [36]:
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for sequences, labels in loader:
            output = model(sequences)
            _, predicted = torch.max(output, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return correct / total

In [37]:
EPOCHS = 10
for epoch in range(EPOCHS):
    train_loss = train(model, train_loader)
    test_acc = evaluate(model, test_loader)
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}")

Epoch 1/10, Loss: 0.9163, Test Accuracy: 0.7698
Epoch 2/10, Loss: 0.3210, Test Accuracy: 0.8184
Epoch 3/10, Loss: 0.2803, Test Accuracy: 0.8037
Epoch 4/10, Loss: 0.2560, Test Accuracy: 0.7521
Epoch 5/10, Loss: 0.2368, Test Accuracy: 0.7744
Epoch 6/10, Loss: 0.2199, Test Accuracy: 0.7509
Epoch 7/10, Loss: 0.2054, Test Accuracy: 0.7310
Epoch 8/10, Loss: 0.1916, Test Accuracy: 0.7284
Epoch 9/10, Loss: 0.1787, Test Accuracy: 0.7222
Epoch 10/10, Loss: 0.1657, Test Accuracy: 0.7219


In [38]:
# Final evaluation
final_accuracy = evaluate(model, test_loader)
print(f"Final Test Accuracy: {final_accuracy:.4f}")

Final Test Accuracy: 0.7219
